<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/Pruning_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prune Bert Model Comparision with Normal Bert Model

In [1]:
!pip install -q transformers datasets

     |████████████████████████████████| 1.9MB 21.9MB/s 
     |████████████████████████████████| 194kB 55.8MB/s 
     |████████████████████████████████| 3.2MB 49.4MB/s 
     |████████████████████████████████| 890kB 53.3MB/s 
     |████████████████████████████████| 245kB 50.0MB/s 
     |████████████████████████████████| 112kB 61.3MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from datasets import load_metric, load_dataset
from tqdm.notebook import tqdm
device =  'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
tokenizer = AutoTokenizer.from_pretrained("huggingface/prunebert-base-uncased-6-finepruned-w-distil-squad")
model = AutoModelForQuestionAnswering.from_pretrained("huggingface/prunebert-base-uncased-6-finepruned-w-distil-squad")

In [4]:
bert_tokenizer = AutoTokenizer.from_pretrained("csarron/bert-base-uncased-squad-v1")
bert_model = AutoModelForQuestionAnswering.from_pretrained("csarron/bert-base-uncased-squad-v1")

In [5]:
def get_result(predictions, references, dataset='squad_v2'):
  squad_metric = load_metric(dataset)
  score = squad_metric.compute(predictions=predictions, references=references)
  return score

def get_validation_data(dataset='squad_v2'):
  datasets = load_dataset(dataset)
  valid_dataset = datasets['validation']
  return valid_dataset

def get_infernece(valid_dataset, model, tokenizer, device):
  predictions=[]
  references=[]
  model.to(device)
  for example in tqdm(valid_dataset):
    inputs = tokenizer(example['question'], example['context'], return_tensors="pt", truncation=True)
    inputs.to(device)
    output = model(**inputs)
    start_index = torch.argmax(output['start_logits'])
    end_index = torch.argmax(output['end_logits'])
    ans_ids = inputs['input_ids'][0][start_index :end_index+1]
    answer = tokenizer.decode(ans_ids)
    answer = tokenizer.clean_up_tokenization(answer).strip()
    no_answer_probability = 1 if len(example['answers']['answer_start'])!=0 else 0
    """
    predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22', 'no_answer_probability': 0.}]
    references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
    """
    pred = {'prediction_text':answer, 'id': example['id']}
    ref = {'answers': example['answers'] , 'id': example['id']}
    predictions.append(pred)
    references.append(ref)
  return predictions, references

In [7]:
def print_results(model, tokenizer, device, dataset_name='squad'):
  validation_dataset = get_validation_data(dataset_name)
  predictions, references = get_infernece(validation_dataset, model, tokenizer, device)
  score = get_result(predictions, references, dataset_name)
  return score

In [8]:
## Inference for bert Model on Colab GPU
## Note: This script for inference is not 100% perfect 
# It does not include sliding window and post processing
print_results(bert_model, bert_tokenizer, device)

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9. Subsequent calls will reuse this data.



{'exact_match': 72.07190160832545, 'f1': 81.26854601807015}


In [10]:
## Inference for prone bert Model on Colab GPU
print_results(model, tokenizer, device)

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9)



{'exact_match': 67.18070009460737, 'f1': 77.45808864399302}


## Performance on Colab GPU:

SQUAD v1 Dataset:

| Model | EM | F1 | Inference Time (10570 examples)| Throughput | Model Size |
| --- | --- | --- | --- | --- | --- |
| Bert | 72.07 | 81.26 | 3mins | 58.72 examples/sec | 417.7mb | 
| PruneBert | 67.18 | 77.46 | 2mins 47 secs| 63.29 exmaples/sec | 417.7mb | 